# Analisis Data FMCG Personal Care
## Data Science Competition - Gelar Rasa 2025

**Objective:** Menganalisis dataset produk FMCG Personal Care untuk mendapatkan insight mendalam terkait:
- Innovation Radar: Positioning produk berdasarkan inovasi
- Trend Forecasting: Prediksi tren penjualan masa depan
- Product Cannibalization: Identifikasi kompetisi internal antar produk

**Dataset:** Sales, Products, Marketing, Reviews

**Approach:** End-to-end data science pipeline dari ingestion hingga business insights

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from datetime import datetime, timedelta
import os
from pathlib import Path
import json

# Stats & ML
from scipy import stats
from scipy.stats import normaltest, jarque_bera, shapiro
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan, acorr_ljungbox
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split, cross_val_score, TimeSeriesSplit
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

warnings.filterwarnings('ignore')

# Reproducibility
SEED = 42
np.random.seed(SEED)

# Package versions for documentation
versions = {
    'pandas': pd.__version__,
    'numpy': np.__version__,
}

try:
    import sklearn
    versions['sklearn'] = sklearn.__version__
except ImportError:
    pass

print("=" * 80)
print("ENVIRONMENT SETUP")
print("=" * 80)
print(f"Random Seed: {SEED}")
print("\nLibrary Versions:")
for lib, ver in versions.items():
    print(f"  {lib:15s}: {ver}")

# Directory structure
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "Gelar_Rasa" / "data" / "fmcg_personalcare" / "fmcg_personalcare"
REPORTS_DIR = BASE_DIR / "reports"
PROFILING_DIR = REPORTS_DIR / "profiling"
EDA_DIR = REPORTS_DIR / "eda"
CLEAN_DATA_DIR = BASE_DIR / "data" / "clean"
ARCHIVE_DIR = BASE_DIR / "archive" / "unneeded"

# Create dirs if needed
for d in [REPORTS_DIR, PROFILING_DIR, EDA_DIR, CLEAN_DATA_DIR, ARCHIVE_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print(f"\nPaths configured:")
print(f"  Data: {DATA_DIR}")
print(f"  Reports: {REPORTS_DIR}")
print(f"  Clean: {CLEAN_DATA_DIR}")
print("\n✓ Setup complete")

ENVIRONMENT SETUP
Random Seed: 42

Library Versions:
  pandas         : 2.3.3
  numpy          : 2.3.4
  sklearn        : 1.7.2

Paths configured:
  Data: c:\Users\Lenovo\OneDrive\Desktop\DATA\Gelar-Rasa\Gelar_Rasa\data\fmcg_personalcare\fmcg_personalcare
  Reports: c:\Users\Lenovo\OneDrive\Desktop\DATA\Gelar-Rasa\reports
  Clean: c:\Users\Lenovo\OneDrive\Desktop\DATA\Gelar-Rasa\data\clean

✓ Setup complete


## 2. Repository Structure Analysis

Before diving into data, let's understand the file organization.

In [2]:
def show_tree(path, max_depth=3, current_depth=0, prefix=""):
    """Display directory tree structure."""
    if current_depth > max_depth:
        return
    
    if not path.exists():
        print(f"{prefix}⚠ Path not found: {path}")
        return
    
    try:
        items = sorted(path.iterdir())
    except PermissionError:
        print(f"{prefix}⚠ Permission denied")
        return
    
    # Filter out hidden and common noise
    items = [i for i in items if not i.name.startswith('.') 
             and i.name not in ['__pycache__', 'node_modules']]
    
    for idx, item in enumerate(items):
        is_last = idx == len(items) - 1
        connector = "└── " if is_last else "├── "
        extension = "    " if is_last else "│   "
        
        if item.is_file():
            size = item.stat().st_size
            size_str = f"{size:,} bytes" if size < 1024 else f"{size/1024:.1f} KB"
            print(f"{prefix}{connector}{item.name} ({size_str})")
        else:
            print(f"{prefix}{connector}{item.name}/")
            show_tree(item, max_depth, current_depth + 1, prefix + extension)

print("Repository Structure:")
print("=" * 80)
print(f"📁 {BASE_DIR.name}/")
show_tree(BASE_DIR, max_depth=2)

# Check if data files exist
print("\n" + "=" * 80)
print("Data Files Check:")
print("=" * 80)
required_files = ['sales.csv', 'products.csv', 'marketing.csv', 'reviews.csv']
missing = []
for fname in required_files:
    fpath = DATA_DIR / fname
    if fpath.exists():
        size = fpath.stat().st_size / 1024
        print(f"✓ {fname:20s} - {size:,.1f} KB")
    else:
        print(f"✗ {fname:20s} - NOT FOUND")
        missing.append(fname)

if missing:
    print(f"\n⚠ Missing files: {', '.join(missing)}")
else:
    print("\n✓ All required files present")

Repository Structure:
📁 Gelar-Rasa/
├── archive/
│   └── unneeded/
├── data/
│   └── clean/
├── DSC2025_Analysis_FMCG_PersonalCare.ipynb (2344.4 KB)
├── DSC2025_Analysis_FMCG_PersonalCare_Fixed.ipynb (48.8 KB)
├── env/
│   ├── etc/
│   │   └── jupyter/
│   ├── Include/
│   ├── Lib/
│   │   └── site-packages/
│   ├── pyvenv.cfg (340 bytes)
│   ├── Scripts/
│   │   ├── activate (1.7 KB)
│   │   ├── activate.bat (1,019 bytes)
│   │   ├── Activate.ps1 (25.6 KB)
│   │   ├── choreo_diagnose.exe (105.9 KB)
│   │   ├── choreo_get_chrome.exe (105.9 KB)
│   │   ├── cygdb.exe (105.9 KB)
│   │   ├── cython.exe (105.9 KB)
│   │   ├── cythonize.exe (105.9 KB)
│   │   ├── deactivate.bat (393 bytes)
│   │   ├── debugpy-adapter.exe (105.9 KB)
│   │   ├── debugpy.exe (105.9 KB)
│   │   ├── f2py.exe (105.9 KB)
│   │   ├── fonttools.exe (105.9 KB)
│   │   ├── httpx.exe (105.9 KB)
│   │   ├── import_pb_to_tensorboard.exe (105.9 KB)
│   │   ├── install_cmdstan.exe (105.9 KB)
│   │   ├── install_cxx_toolchai

## 3. Data Loading & Initial Profiling

Load all datasets and get quick stats about shape, types, and missing values.

In [3]:
print("=" * 80)
print("LOADING DATASETS")
print("=" * 80)

# Load with error handling
datasets = {}
for fname in ['sales', 'products', 'marketing', 'reviews']:
    fpath = DATA_DIR / f"{fname}.csv"
    try:
        df = pd.read_csv(fpath)
        datasets[fname] = df
        print(f"✓ Loaded {fname:12s}: {len(df):6,} rows × {len(df.columns):2} cols")
    except FileNotFoundError:
        print(f"✗ Failed to load {fname}.csv - file not found")
    except Exception as e:
        print(f"✗ Failed to load {fname}.csv - {str(e)}")

# Unpack for convenience
df_sales = datasets.get('sales')
df_products = datasets.get('products')
df_marketing = datasets.get('marketing')
df_reviews = datasets.get('reviews')

# Show previews
for name, df in datasets.items():
    if df is not None:
        print(f"\n{'=' * 80}")
        print(f"PREVIEW: {name.upper()}")
        print("=" * 80)
        print(df.head(3))
        print(f"\nColumns: {list(df.columns)}")

LOADING DATASETS
✓ Loaded sales       : 1,000,000 rows × 10 cols
✓ Loaded products    :     15 rows ×  7 cols
✓ Loaded marketing   :     20 rows ×  8 cols
✓ Loaded reviews     : 10,000 rows ×  7 cols

PREVIEW: SALES
                         transaction_id        date product_id     region  \
0  fa43024c-7c61-4e2b-bfd1-84915e86c4f7  2024-02-04      PC001   Semarang   
1  cc02b941-61b7-4e37-b385-ecfc2fe2a0b0  2022-05-31      PC004      Medan   
2  378b66ac-8b18-43e3-8b91-cfbb0ee1d346  2020-10-24      PC013  Palembang   

     channel  units_sold  avg_price  discount_pct    revenue  \
0     Shopee           4   31721.61             5  126886.45   
1  Tokopedia           4   23148.20             0   92592.80   
2   Alfamart           5   25396.69            20  126983.43   

   days_since_launch  
0               1450  
1                395  
2              -1178  

Columns: ['transaction_id', 'date', 'product_id', 'region', 'channel', 'units_sold', 'avg_price', 'discount_pct', 'revenue', 

In [4]:
def profile_dataframe(df, name):
    """Generate comprehensive profile of a dataframe."""
    profile = {
        'dataset': name,
        'rows': len(df),
        'columns': len(df.columns),
        'memory_mb': df.memory_usage(deep=True).sum() / 1024**2,
        'duplicates': df.duplicated().sum(),
        'missing_total': df.isnull().sum().sum(),
        'dtypes': df.dtypes.value_counts().to_dict()
    }
    
    # Column-level stats
    col_stats = []
    for col in df.columns:
        missing = df[col].isnull().sum()
        unique = df[col].nunique()
        col_stats.append({
            'column': col,
            'type': str(df[col].dtype),
            'missing': missing,
            'missing_pct': 100 * missing / len(df),
            'unique': unique,
            'unique_pct': 100 * unique / len(df)
        })
    
    profile['columns_detail'] = pd.DataFrame(col_stats)
    return profile

print("=" * 80)
print("DATA PROFILING")
print("=" * 80)

profiles = {}
for name, df in datasets.items():
    if df is not None:
        profiles[name] = profile_dataframe(df, name)

# Display summary
for name, prof in profiles.items():
    print(f"\n{name.upper()}:")
    print(f"  Shape: {prof['rows']:,} rows × {prof['columns']} cols")
    print(f"  Memory: {prof['memory_mb']:.2f} MB")
    print(f"  Duplicates: {prof['duplicates']:,} ({100*prof['duplicates']/prof['rows']:.2f}%)")
    print(f"  Missing values: {prof['missing_total']:,}")
    print(f"  Data types: {prof['dtypes']}")
    
    # Show columns with missing data
    missing_cols = prof['columns_detail'][prof['columns_detail']['missing'] > 0]
    if len(missing_cols) > 0:
        print(f"\n  Columns with missing data:")
        for _, row in missing_cols.iterrows():
            print(f"    - {row['column']:25s}: {row['missing']:5,} ({row['missing_pct']:5.2f}%)")

# Save profiling reports
for name, prof in profiles.items():
    report_path = PROFILING_DIR / f"{name}_profile.csv"
    prof['columns_detail'].to_csv(report_path, index=False)
    print(f"\n✓ Saved profile: {report_path}")

DATA PROFILING

SALES:
  Shape: 1,000,000 rows × 10 cols
  Memory: 374.70 MB
  Duplicates: 0 (0.00%)
  Missing values: 0
  Data types: {dtype('O'): 5, dtype('int64'): 3, dtype('float64'): 2}

PRODUCTS:
  Shape: 15 rows × 7 cols
  Memory: 0.01 MB
  Duplicates: 0 (0.00%)
  Missing values: 0
  Data types: {dtype('O'): 5, dtype('int64'): 2}

MARKETING:
  Shape: 20 rows × 8 cols
  Memory: 0.01 MB
  Duplicates: 0 (0.00%)
  Missing values: 0
  Data types: {dtype('O'): 6, dtype('int64'): 1, dtype('float64'): 1}

REVIEWS:
  Shape: 10,000 rows × 7 cols
  Memory: 4.06 MB
  Duplicates: 0 (0.00%)
  Missing values: 0
  Data types: {dtype('O'): 6, dtype('float64'): 1}

✓ Saved profile: c:\Users\Lenovo\OneDrive\Desktop\DATA\Gelar-Rasa\reports\profiling\sales_profile.csv

✓ Saved profile: c:\Users\Lenovo\OneDrive\Desktop\DATA\Gelar-Rasa\reports\profiling\products_profile.csv

✓ Saved profile: c:\Users\Lenovo\OneDrive\Desktop\DATA\Gelar-Rasa\reports\profiling\marketing_profile.csv

✓ Saved profile: c:\U

## 4. Data Cleaning

Handle missing values, duplicates, and outliers systematically.

In [5]:
print("=" * 80)
print("DATA CLEANING")
print("=" * 80)

cleaning_log = []

# Step 1: Handle duplicates
print("\nStep 1: Removing duplicates")
for name in ['sales', 'products', 'marketing', 'reviews']:
    df = datasets.get(name)
    if df is not None:
        before = len(df)
        df = df.drop_duplicates()
        after = len(df)
        removed = before - after
        if removed > 0:
            print(f"  {name:12s}: removed {removed:,} duplicates ({100*removed/before:.2f}%)")
            cleaning_log.append(f"{name}: removed {removed} duplicate rows")
        else:
            print(f"  {name:12s}: no duplicates")
        datasets[name] = df

# Step 2: Convert date columns
print("\nStep 2: Parsing date columns")
if df_sales is not None and 'date' in df_sales.columns:
    df_sales['date'] = pd.to_datetime(df_sales['date'], errors='coerce')
    invalid = df_sales['date'].isnull().sum()
    if invalid > 0:
        print(f"  ⚠ Found {invalid} invalid dates in sales.date")
    else:
        print(f"  ✓ sales.date parsed successfully")
    datasets['sales'] = df_sales

if df_marketing is not None and 'campaign_start' in df_marketing.columns:
    df_marketing['campaign_start'] = pd.to_datetime(df_marketing['campaign_start'], errors='coerce')
    print(f"  ✓ marketing.campaign_start parsed")
    datasets['marketing'] = df_marketing

# Step 3: Handle missing values
print("\nStep 3: Handling missing values")

# Sales: drop rows with missing critical fields
if df_sales is not None:
    critical_cols = ['product_id', 'quantity', 'revenue']
    existing_critical = [c for c in critical_cols if c in df_sales.columns]
    before = len(df_sales)
    df_sales = df_sales.dropna(subset=existing_critical)
    after = len(df_sales)
    removed = before - after
    if removed > 0:
        print(f"  sales: dropped {removed} rows with missing critical fields")
        cleaning_log.append(f"sales: dropped {removed} rows with missing {existing_critical}")
    datasets['sales'] = df_sales

# Products: fill categorical with 'Unknown', numeric with median
if df_products is not None:
    for col in df_products.columns:
        if df_products[col].isnull().sum() > 0:
            if df_products[col].dtype == 'object':
                df_products[col].fillna('Unknown', inplace=True)
                print(f"  products.{col}: filled {df_products[col].isnull().sum()} missing with 'Unknown'")
            else:
                median_val = df_products[col].median()
                df_products[col].fillna(median_val, inplace=True)
                print(f"  products.{col}: filled missing with median={median_val:.2f}")
    datasets['products'] = df_products

# Reviews: drop if missing rating or text
if df_reviews is not None:
    review_critical = ['rating']
    existing_review = [c for c in review_critical if c in df_reviews.columns]
    before = len(df_reviews)
    df_reviews = df_reviews.dropna(subset=existing_review)
    after = len(df_reviews)
    removed = before - after
    if removed > 0:
        print(f"  reviews: dropped {removed} rows with missing {existing_review}")
        cleaning_log.append(f"reviews: dropped {removed} rows with missing rating")
    datasets['reviews'] = df_reviews

# Step 4: Data type optimization
print("\nStep 4: Optimizing data types")
for name, df in datasets.items():
    if df is not None:
        before_mem = df.memory_usage(deep=True).sum() / 1024**2
        
        # Downcast numeric columns
        for col in df.select_dtypes(include=['int64']).columns:
            df[col] = pd.to_numeric(df[col], downcast='integer')
        
        for col in df.select_dtypes(include=['float64']).columns:
            df[col] = pd.to_numeric(df[col], downcast='float')
        
        # Convert low-cardinality strings to category
        for col in df.select_dtypes(include=['object']).columns:
            if df[col].nunique() / len(df) < 0.5:  # less than 50% unique
                df[col] = df[col].astype('category')
        
        after_mem = df.memory_usage(deep=True).sum() / 1024**2
        savings = 100 * (1 - after_mem/before_mem)
        print(f"  {name:12s}: {before_mem:.2f}MB → {after_mem:.2f}MB (saved {savings:.1f}%)")
        datasets[name] = df

print("\n✓ Cleaning complete")

DATA CLEANING

Step 1: Removing duplicates
  sales       : no duplicates
  products    : no duplicates
  marketing   : no duplicates
  reviews     : no duplicates

Step 2: Parsing date columns
  ✓ sales.date parsed successfully

Step 3: Handling missing values

Step 4: Optimizing data types
  sales       : 318.43MB → 118.26MB (saved 62.9%)
  products    : 0.01MB → 0.01MB (saved 2.7%)
  marketing   : 0.01MB → 0.01MB (saved 10.3%)
  reviews     : 4.06MB → 0.91MB (saved 77.6%)

✓ Cleaning complete


In [6]:
# Step 5: Outlier detection and handling
print("\nStep 5: Outlier detection (IQR method)")

def detect_outliers_iqr(series, multiplier=1.5):
    """Detect outliers using IQR method."""
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - multiplier * IQR
    upper = Q3 + multiplier * IQR
    return (series < lower) | (series > upper)

# Check sales for outliers in quantity and revenue
if df_sales is not None:
    for col in ['quantity', 'revenue']:
        if col in df_sales.columns:
            outliers = detect_outliers_iqr(df_sales[col])
            n_outliers = outliers.sum()
            if n_outliers > 0:
                pct = 100 * n_outliers / len(df_sales)
                print(f"  sales.{col}: {n_outliers:,} outliers ({pct:.2f}%)")
                
                # Cap outliers at 99th percentile instead of removing
                cap_value = df_sales[col].quantile(0.99)
                df_sales.loc[outliers, col] = cap_value
                print(f"    → Capped at 99th percentile: {cap_value:.2f}")
                cleaning_log.append(f"sales.{col}: capped {n_outliers} outliers")
    datasets['sales'] = df_sales

print("\n✓ Outlier handling complete")

# Unpack cleaned data
df_sales = datasets.get('sales')
df_products = datasets.get('products')
df_marketing = datasets.get('marketing')
df_reviews = datasets.get('reviews')


Step 5: Outlier detection (IQR method)
  sales.revenue: 15,620 outliers (1.56%)
    → Capped at 99th percentile: 289770.45

✓ Outlier handling complete


## 5. Feature Engineering

Create domain-specific features for FMCG analysis.

In [7]:
print("=" * 80)
print("FEATURE ENGINEERING")
print("=" * 80)

# Merge datasets
print("\nMerging datasets...")
if df_sales is not None and df_products is not None:
    df_merged = df_sales.merge(df_products, on='product_id', how='left')
    print(f"  ✓ Merged sales + products: {len(df_merged):,} rows")
    
    if df_reviews is not None:
        # Aggregate reviews per product
        review_agg = df_reviews.groupby('product_id').agg({
            'rating': ['mean', 'count', 'std'],
        }).reset_index()
        review_agg.columns = ['product_id', 'avg_rating', 'review_count', 'rating_std']
        review_agg['rating_std'].fillna(0, inplace=True)  # handle single review case
        
        df_merged = df_merged.merge(review_agg, on='product_id', how='left')
        df_merged['avg_rating'].fillna(df_merged['avg_rating'].median(), inplace=True)
        df_merged['review_count'].fillna(0, inplace=True)
        df_merged['rating_std'].fillna(0, inplace=True)
        print(f"  ✓ Added review aggregates")
    
    # Create FMCG-specific features
    print("\nEngineering FMCG features...")
    
    # Price per unit
    if 'revenue' in df_merged.columns and 'quantity' in df_merged.columns:
        df_merged['price_per_unit'] = df_merged['revenue'] / df_merged['quantity']
        print("  ✓ price_per_unit")
    
    # Temporal features
    if 'date' in df_merged.columns:
        df_merged['year'] = df_merged['date'].dt.year
        df_merged['month'] = df_merged['date'].dt.month
        df_merged['quarter'] = df_merged['date'].dt.quarter
        df_merged['day_of_week'] = df_merged['date'].dt.dayofweek
        df_merged['is_weekend'] = (df_merged['day_of_week'] >= 5).astype(int)
        df_merged['week_of_year'] = df_merged['date'].dt.isocalendar().week
        print("  ✓ Temporal features (year, month, quarter, etc.)")
    
    # Product age (if launch_date exists)
    if 'launch_date' in df_merged.columns:
        df_merged['launch_date'] = pd.to_datetime(df_merged['launch_date'], errors='coerce')
        df_merged['product_age_days'] = (df_merged['date'] - df_merged['launch_date']).dt.days
        print("  ✓ product_age_days")
    
    # Brand performance metrics
    if 'brand' in df_merged.columns:
        brand_revenue = df_merged.groupby('brand')['revenue'].transform('sum')
        df_merged['brand_total_revenue'] = brand_revenue
        df_merged['revenue_share_in_brand'] = df_merged['revenue'] / brand_revenue
        print("  ✓ Brand performance metrics")
    
    # Category performance
    if 'category' in df_merged.columns:
        cat_revenue = df_merged.groupby('category')['revenue'].transform('sum')
        df_merged['category_total_revenue'] = cat_revenue
        df_merged['revenue_share_in_category'] = df_merged['revenue'] / cat_revenue
        print("  ✓ Category performance metrics")
    
    print(f"\n✓ Final merged dataset: {len(df_merged):,} rows × {len(df_merged.columns)} cols")
else:
    print("⚠ Cannot merge - missing required datasets")
    df_merged = None

FEATURE ENGINEERING

Merging datasets...
  ✓ Merged sales + products: 1,000,000 rows
  ✓ Added review aggregates

Engineering FMCG features...
  ✓ Temporal features (year, month, quarter, etc.)
  ✓ product_age_days
  ✓ Brand performance metrics

✓ Final merged dataset: 1,000,000 rows × 28 cols


In [8]:
# Lag and rolling features for time series
print("\nCreating time series features (lag & rolling)...")

if df_merged is not None and 'date' in df_merged.columns:
    # Sort by product and date
    df_merged = df_merged.sort_values(['product_id', 'date'])
    
    # Create lag features
    for lag in [1, 7, 30]:
        df_merged[f'revenue_lag_{lag}'] = df_merged.groupby('product_id')['revenue'].shift(lag)
        df_merged[f'quantity_lag_{lag}'] = df_merged.groupby('product_id')['quantity'].shift(lag)
    
    print("  ✓ Lag features (1, 7, 30 days)")
    
    # Rolling averages
    for window in [7, 30]:
        df_merged[f'revenue_rolling_{window}'] = df_merged.groupby('product_id')['revenue'].transform(
            lambda x: x.rolling(window, min_periods=1).mean()
        )
        df_merged[f'quantity_rolling_{window}'] = df_merged.groupby('product_id')['quantity'].transform(
            lambda x: x.rolling(window, min_periods=1).mean()
        )
    
    print("  ✓ Rolling averages (7, 30 days)")
    
    # Trend indicators
    df_merged['revenue_trend_7d'] = (
        df_merged['revenue_rolling_7'] / df_merged['revenue_lag_7'] - 1
    ) * 100
    df_merged['revenue_trend_7d'].fillna(0, inplace=True)
    
    print("  ✓ Trend indicators")
    print(f"\n✓ Time series features complete")
else:
    print("⚠ Skipping time series features - date column not found")


Creating time series features (lag & rolling)...


KeyError: 'Column not found: quantity'

## 6. Exploratory Data Analysis

Visual exploration of patterns, trends, and relationships.

In [ ]:
print("=" * 80)
print("EXPLORATORY DATA ANALYSIS")
print("=" * 80)

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

if df_merged is not None:
    # Revenue distribution
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    axes[0].hist(df_merged['revenue'], bins=50, edgecolor='black', alpha=0.7)
    axes[0].set_xlabel('Revenue')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title('Revenue Distribution')
    axes[0].grid(alpha=0.3)
    
    axes[1].hist(np.log1p(df_merged['revenue']), bins=50, edgecolor='black', alpha=0.7, color='orange')
    axes[1].set_xlabel('Log(Revenue + 1)')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Revenue Distribution (Log Scale)')
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(EDA_DIR / 'revenue_distribution.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✓ Saved: revenue_distribution.png")
    
    # Time series of revenue
    if 'date' in df_merged.columns:
        daily_revenue = df_merged.groupby('date')['revenue'].sum().reset_index()
        
        fig, ax = plt.subplots(figsize=(14, 5))
        ax.plot(daily_revenue['date'], daily_revenue['revenue'], linewidth=1.5)
        ax.set_xlabel('Date')
        ax.set_ylabel('Total Revenue')
        ax.set_title('Revenue Over Time')
        ax.grid(alpha=0.3)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(EDA_DIR / 'revenue_timeseries.png', dpi=150, bbox_inches='tight')
        plt.show()
        print("✓ Saved: revenue_timeseries.png")
    
    # Top products by revenue
    if 'product_name' in df_merged.columns:
        top_products = df_merged.groupby('product_name')['revenue'].sum().nlargest(15)
        
        fig, ax = plt.subplots(figsize=(10, 6))
        top_products.plot(kind='barh', ax=ax, color='steelblue')
        ax.set_xlabel('Total Revenue')
        ax.set_ylabel('Product')
        ax.set_title('Top 15 Products by Revenue')
        ax.grid(axis='x', alpha=0.3)
        plt.tight_layout()
        plt.savefig(EDA_DIR / 'top_products.png', dpi=150, bbox_inches='tight')
        plt.show()
        print("✓ Saved: top_products.png")
    
    # Category analysis
    if 'category' in df_merged.columns:
        category_stats = df_merged.groupby('category').agg({
            'revenue': 'sum',
            'quantity': 'sum',
            'product_id': 'nunique'
        }).round(2)
        category_stats.columns = ['Total Revenue', 'Total Quantity', 'Unique Products']
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        category_stats['Total Revenue'].sort_values().plot(
            kind='barh', ax=axes[0], color='coral'
        )
        axes[0].set_xlabel('Total Revenue')
        axes[0].set_title('Revenue by Category')
        axes[0].grid(axis='x', alpha=0.3)
        
        category_stats['Unique Products'].sort_values().plot(
            kind='barh', ax=axes[1], color='lightseagreen'
        )
        axes[1].set_xlabel('Number of Products')
        axes[1].set_title('Product Count by Category')
        axes[1].grid(axis='x', alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(EDA_DIR / 'category_analysis.png', dpi=150, bbox_inches='tight')
        plt.show()
        print("✓ Saved: category_analysis.png")

print("\n✓ EDA visualizations complete")

In [ ]:
# Correlation heatmap
print("\nCreating correlation heatmap...")

if df_merged is not None:
    # Select numeric columns for correlation
    numeric_cols = df_merged.select_dtypes(include=[np.number]).columns
    # Filter out ID columns and date-derived integers
    analysis_cols = [c for c in numeric_cols 
                     if not c.endswith('_id') 
                     and c not in ['year', 'month', 'quarter', 'week_of_year', 'day_of_week']]
    
    if len(analysis_cols) > 1:
        corr_matrix = df_merged[analysis_cols].corr()
        
        # Plot
        fig, ax = plt.subplots(figsize=(12, 10))
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
        sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', 
                    cmap='coolwarm', center=0, square=True,
                    linewidths=0.5, cbar_kws={"shrink": 0.8}, ax=ax)
        ax.set_title('Feature Correlation Matrix', fontsize=14, pad=20)
        plt.tight_layout()
        plt.savefig(EDA_DIR / 'correlation_heatmap.png', dpi=150, bbox_inches='tight')
        plt.show()
        print("✓ Saved: correlation_heatmap.png")
        
        # Show top correlations with revenue
        if 'revenue' in corr_matrix.columns:
            revenue_corr = corr_matrix['revenue'].abs().sort_values(ascending=False)
            print("\nTop correlations with revenue:")
            for feat, val in revenue_corr.head(10).items():
                if feat != 'revenue':
                    print(f"  {feat:30s}: {val:.3f}")
    else:
        print("⚠ Not enough numeric columns for correlation analysis")
else:
    print("⚠ Merged dataset not available")

## 7. Modeling & Evaluation

Build predictive models for revenue forecasting.

In [ ]:
print("=" * 80)
print("MODELING & EVALUATION")
print("=" * 80)

if df_merged is not None and 'revenue' in df_merged.columns:
    # Prepare features
    print("\nPreparing features...")
    
    # Select feature columns (exclude target and non-predictive columns)
    exclude_cols = ['revenue', 'date', 'product_id', 'product_name', 'brand', 
                    'category', 'launch_date']
    feature_cols = [c for c in df_merged.columns 
                    if c not in exclude_cols 
                    and df_merged[c].dtype in [np.float64, np.float32, np.int64, np.int32]]
    
    # Drop rows with NaN in features or target
    model_data = df_merged[feature_cols + ['revenue']].dropna()
    
    X = model_data[feature_cols]
    y = model_data['revenue']
    
    print(f"  Features: {len(feature_cols)} columns")
    print(f"  Samples: {len(X):,} rows")
    print(f"  Target: revenue (mean={y.mean():.2f}, std={y.std():.2f})")
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=SEED
    )
    print(f"\n  Train: {len(X_train):,} samples")
    print(f"  Test:  {len(X_test):,} samples")
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    print("  ✓ Features scaled")
else:
    print("⚠ Cannot proceed with modeling - data not available")
    X_train, X_test, y_train, y_test = None, None, None, None

In [ ]:
# Train baseline models
print("\nTraining models...")

if X_train is not None:
    models = {
        'Linear Regression': LinearRegression(),
        'Ridge': Ridge(alpha=1.0, random_state=SEED),
        'Lasso': Lasso(alpha=1.0, random_state=SEED),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=SEED, n_jobs=-1),
        'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=SEED)
    }
    
    results = []
    trained_models = {}
    
    for name, model in models.items():
        print(f"\n  Training {name}...")
        
        # Train
        model.fit(X_train_scaled, y_train)
        
        # Predict
        y_pred_train = model.predict(X_train_scaled)
        y_pred_test = model.predict(X_test_scaled)
        
        # Evaluate
        train_r2 = r2_score(y_train, y_pred_train)
        test_r2 = r2_score(y_test, y_pred_test)
        test_mae = mean_absolute_error(y_test, y_pred_test)
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
        test_mape = mean_absolute_percentage_error(y_test, y_pred_test) * 100
        
        results.append({
            'Model': name,
            'Train R²': train_r2,
            'Test R²': test_r2,
            'MAE': test_mae,
            'RMSE': test_rmse,
            'MAPE (%)': test_mape
        })
        
        trained_models[name] = model
        
        print(f"    Train R²: {train_r2:.4f}")
        print(f"    Test R²:  {test_r2:.4f}")
        print(f"    MAE:      {test_mae:.2f}")
        print(f"    RMSE:     {test_rmse:.2f}")
        print(f"    MAPE:     {test_mape:.2f}%")
    
    # Results summary
    results_df = pd.DataFrame(results)
    print("\n" + "=" * 80)
    print("MODEL COMPARISON")
    print("=" * 80)
    print(results_df.to_string(index=False))
    
    # Save results
    results_df.to_csv(REPORTS_DIR / 'model_results.csv', index=False)
    print("\n✓ Saved: model_results.csv")
    
    # Best model
    best_model_name = results_df.loc[results_df['Test R²'].idxmax(), 'Model']
    best_model = trained_models[best_model_name]
    print(f"\n✓ Best model: {best_model_name}")
else:
    print("⚠ Skipping model training - data not available")
    best_model = None
    best_model_name = None

In [ ]:
# Model diagnostics for linear regression
if X_train is not None and 'Linear Regression' in trained_models:
    print("\n" + "=" * 80)
    print("MODEL DIAGNOSTICS - LINEAR REGRESSION")
    print("=" * 80)
    
    lr_model = trained_models['Linear Regression']
    y_pred = lr_model.predict(X_test_scaled)
    residuals = y_test - y_pred
    
    # Residual plots
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Residuals vs Fitted
    axes[0, 0].scatter(y_pred, residuals, alpha=0.5, s=10)
    axes[0, 0].axhline(y=0, color='r', linestyle='--', linewidth=2)
    axes[0, 0].set_xlabel('Fitted values')
    axes[0, 0].set_ylabel('Residuals')
    axes[0, 0].set_title('Residuals vs Fitted')
    axes[0, 0].grid(alpha=0.3)
    
    # 2. Q-Q plot
    stats.probplot(residuals, dist="norm", plot=axes[0, 1])
    axes[0, 1].set_title('Normal Q-Q Plot')
    axes[0, 1].grid(alpha=0.3)
    
    # 3. Scale-Location
    standardized_residuals = residuals / np.std(residuals)
    axes[1, 0].scatter(y_pred, np.sqrt(np.abs(standardized_residuals)), alpha=0.5, s=10)
    axes[1, 0].set_xlabel('Fitted values')
    axes[1, 0].set_ylabel('√|Standardized residuals|')
    axes[1, 0].set_title('Scale-Location')
    axes[1, 0].grid(alpha=0.3)
    
    # 4. Residual histogram
    axes[1, 1].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
    axes[1, 1].set_xlabel('Residuals')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].set_title('Residual Distribution')
    axes[1, 1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(REPORTS_DIR / 'model_diagnostics.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("\n✓ Saved: model_diagnostics.png")
    
    # Statistical tests
    print("\nStatistical Tests:")
    
    # Normality test
    _, p_shapiro = shapiro(residuals[:5000] if len(residuals) > 5000 else residuals)
    print(f"  Shapiro-Wilk (normality): p={p_shapiro:.4f}")
    if p_shapiro > 0.05:
        print("    ✓ Residuals appear normally distributed")
    else:
        print("    ⚠ Residuals may not be normally distributed")
    
    # Durbin-Watson (autocorrelation)
    dw_stat = durbin_watson(residuals)
    print(f"  Durbin-Watson (autocorrelation): {dw_stat:.4f}")
    if 1.5 < dw_stat < 2.5:
        print("    ✓ No significant autocorrelation")
    else:
        print("    ⚠ Possible autocorrelation detected")

In [ ]:
# Feature importance
if best_model is not None and hasattr(best_model, 'feature_importances_'):
    print("\n" + "=" * 80)
    print(f"FEATURE IMPORTANCE - {best_model_name}")
    print("=" * 80)
    
    importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\nTop 15 Most Important Features:")
    for idx, row in importance.head(15).iterrows():
        print(f"  {row['feature']:30s}: {row['importance']:.4f}")
    
    # Plot
    fig, ax = plt.subplots(figsize=(10, 8))
    importance.head(20).plot(x='feature', y='importance', kind='barh', ax=ax, color='steelblue')
    ax.set_xlabel('Importance')
    ax.set_ylabel('Feature')
    ax.set_title(f'Top 20 Feature Importances - {best_model_name}')
    ax.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.savefig(REPORTS_DIR / 'feature_importance.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("\n✓ Saved: feature_importance.png")
    
elif best_model is not None and hasattr(best_model, 'coef_'):
    print("\n" + "=" * 80)
    print(f"FEATURE COEFFICIENTS - {best_model_name}")
    print("=" * 80)
    
    coeffs = pd.DataFrame({
        'feature': feature_cols,
        'coefficient': best_model.coef_
    }).sort_values('coefficient', key=abs, ascending=False)
    
    print("\nTop 15 Coefficients (by absolute value):")
    for idx, row in coeffs.head(15).iterrows():
        print(f"  {row['feature']:30s}: {row['coefficient']:+.4f}")
    
    # Plot
    fig, ax = plt.subplots(figsize=(10, 8))
    top_coeffs = coeffs.head(20)
    colors = ['green' if x > 0 else 'red' for x in top_coeffs['coefficient']]
    ax.barh(top_coeffs['feature'], top_coeffs['coefficient'], color=colors, alpha=0.7)
    ax.set_xlabel('Coefficient')
    ax.set_ylabel('Feature')
    ax.set_title(f'Top 20 Feature Coefficients - {best_model_name}')
    ax.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
    ax.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.savefig(REPORTS_DIR / 'feature_coefficients.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("\n✓ Saved: feature_coefficients.png")

## 8. Bias & Data Validity Analysis

Assess potential biases and data quality issues.

In [ ]:
print("=" * 80)
print("BIAS & DATA VALIDITY ANALYSIS")
print("=" * 80)

if df_merged is not None:
    # Check for sampling bias
    print("\n1. Temporal Coverage:")
    if 'date' in df_merged.columns:
        date_range = df_merged['date'].max() - df_merged['date'].min()
        print(f"   Date range: {df_merged['date'].min()} to {df_merged['date'].max()}")
        print(f"   Coverage: {date_range.days} days")
        
        # Check for gaps
        daily_counts = df_merged.groupby(df_merged['date'].dt.date).size()
        if daily_counts.std() / daily_counts.mean() > 0.5:
            print("   ⚠ High variance in daily transaction counts - possible sampling bias")
        else:
            print("   ✓ Relatively consistent daily coverage")
    
    # Check brand representation
    print("\n2. Brand Representation:")
    if 'brand' in df_merged.columns:
        brand_dist = df_merged['brand'].value_counts()
        print(f"   Unique brands: {len(brand_dist)}")
        print(f"   Top brand share: {100*brand_dist.iloc[0]/len(df_merged):.1f}%")
        if brand_dist.iloc[0] / len(df_merged) > 0.5:
            print("   ⚠ Dataset heavily skewed toward one brand")
        else:
            print("   ✓ Reasonable brand diversity")
    
    # Check category coverage
    print("\n3. Category Coverage:")
    if 'category' in df_merged.columns:
        cat_dist = df_merged['category'].value_counts()
        print(f"   Unique categories: {len(cat_dist)}")
        print(f"   Category distribution:")
        for cat, cnt in cat_dist.head(10).items():
            print(f"     {cat:25s}: {cnt:7,} ({100*cnt/len(df_merged):5.1f}%)")
    
    # Check for unrealistic values
    print("\n4. Value Range Checks:")
    if 'revenue' in df_merged.columns:
        neg_revenue = (df_merged['revenue'] < 0).sum()
        if neg_revenue > 0:
            print(f"   ⚠ Found {neg_revenue} negative revenue values")
        else:
            print("   ✓ No negative revenue values")
    
    if 'quantity' in df_merged.columns:
        neg_qty = (df_merged['quantity'] < 0).sum()
        zero_qty = (df_merged['quantity'] == 0).sum()
        if neg_qty > 0:
            print(f"   ⚠ Found {neg_qty} negative quantity values")
        if zero_qty > 0:
            print(f"   ⚠ Found {zero_qty} zero quantity values")
        if neg_qty == 0 and zero_qty == 0:
            print("   ✓ All quantity values are positive")
    
    # Class imbalance check
    print("\n5. Class Balance (if applicable):")
    categorical_cols = df_merged.select_dtypes(include=['object', 'category']).columns
    for col in categorical_cols[:5]:  # Check first 5 categorical columns
        dist = df_merged[col].value_counts()
        if len(dist) > 1:
            imbalance_ratio = dist.iloc[0] / dist.iloc[-1]
            if imbalance_ratio > 10:
                print(f"   ⚠ {col}: High imbalance (ratio={imbalance_ratio:.1f})")
            else:
                print(f"   ✓ {col}: Acceptable balance (ratio={imbalance_ratio:.1f})")

print("\n✓ Bias analysis complete")

## 9. Summary & Business Insights

Key findings and actionable recommendations.

In [ ]:
print("=" * 80)
print("ANALYSIS SUMMARY & BUSINESS INSIGHTS")
print("=" * 80)

insights = []

# Data quality insights
print("\n1. DATA QUALITY")
if df_merged is not None:
    insights.append(f"- Analyzed {len(df_merged):,} sales transactions")
    if 'product_id' in df_merged.columns:
        insights.append(f"- Covering {df_merged['product_id'].nunique():,} unique products")
    if 'date' in df_merged.columns:
        date_range = (df_merged['date'].max() - df_merged['date'].min()).days
        insights.append(f"- Time period: {date_range} days")

for insight in insights:
    print(insight)

# Top performers
if df_merged is not None:
    print("\n2. TOP PERFORMERS")
    
    if 'product_name' in df_merged.columns:
        top_prod = df_merged.groupby('product_name')['revenue'].sum().nlargest(5)
        print("\nTop 5 Products by Revenue:")
        for idx, (prod, rev) in enumerate(top_prod.items(), 1):
            print(f"  {idx}. {prod}: ${rev:,.2f}")
    
    if 'brand' in df_merged.columns:
        top_brand = df_merged.groupby('brand')['revenue'].sum().nlargest(5)
        print("\nTop 5 Brands by Revenue:")
        for idx, (brand, rev) in enumerate(top_brand.items(), 1):
            print(f"  {idx}. {brand}: ${rev:,.2f}")

# Model performance summary
if best_model_name is not None:
    print("\n3. PREDICTIVE MODEL")
    print(f"- Best performing model: {best_model_name}")
    best_metrics = results_df[results_df['Model'] == best_model_name].iloc[0]
    print(f"- Test R²: {best_metrics['Test R²']:.4f}")
    print(f"- MAPE: {best_metrics['MAPE (%)']:.2f}%")
    print(f"- Model explains {100*best_metrics['Test R²']:.1f}% of revenue variance")

# Recommendations
print("\n4. RECOMMENDATIONS")
recommendations = [
    "Focus marketing budget on top-performing product categories",
    "Investigate underperforming brands for potential improvements",
    "Monitor seasonal trends for inventory optimization",
    "Consider A/B testing on pricing strategies for mid-tier products",
    "Develop loyalty programs for high-value customer segments"
]

for idx, rec in enumerate(recommendations, 1):
    print(f"  {idx}. {rec}")

# Save summary
summary_report = {
    'analysis_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'data_summary': insights,
    'best_model': best_model_name if best_model_name else 'N/A',
    'recommendations': recommendations
}

with open(REPORTS_DIR / 'analysis_summary.json', 'w') as f:
    json.dump(summary_report, f, indent=2)

print("\n" + "=" * 80)
print("✓ Analysis complete!")
print(f"✓ All reports saved to: {REPORTS_DIR}")
print("=" * 80)